In [2]:
import pandas as pd
import numpy as np


In [3]:
X=pd.read_csv('pheno.csv')
Y=pd.read_csv('image.csv')
label=pd.read_csv('label1.csv')

In [4]:
import numpy as np
from scipy.linalg import svd

def dca_fuse(X, Y, label):
    p, n = X.shape
    if Y.shape[1] != n:
        raise ValueError('X and Y must have the same number of columns (samples).')
    elif len(label) != n:
        raise ValueError('The length of the label must be equal to the number of samples.')
    elif n == 1:
        raise ValueError('X and Y must have more than one column (samples)')

    q = Y.shape[0]

    # Calculate mean vectors for each class and for all training data
    classes = np.unique(label)
    print(classes)
    c = len(classes)
    cellX = [X[:, label == cls] for cls in classes]
    cellY = [Y[:, label == cls] for cls in classes]
    nSample = np.array([len(cell) for cell in cellX])

    meanX = np.mean(X, axis=1)
    meanY = np.mean(Y, axis=1)

    classMeanX = np.array([np.mean(cell, axis=1) for cell in cellX])
    classMeanY = np.array([np.mean(cell, axis=1) for cell in cellY])

    PhibX = np.zeros((p, c))
    PhibY = np.zeros((q, c))

    for i in range(c):
        PhibX[:, i] = np.sqrt(nSample[i]) * (classMeanX[:, i] - meanX)
        PhibY[:, i] = np.sqrt(nSample[i]) * (classMeanY[:, i] - meanY)

    # Diagonalize the between-class scatter matrix (Sb) for X
    artSbx = np.dot(PhibX.T, PhibX)
    eigVals, eigVecs = np.linalg.eig(artSbx)
    eigVals = np.abs(eigVals)
    zeroEigIdx = np.where(eigVals / np.max(eigVals) < 1e-6)
    eigVals = np.delete(eigVals, zeroEigIdx)
    eigVecs = np.delete(eigVecs, zeroEigIdx, axis=1)
    idx = np.argsort(eigVals)[::-1]
    eigVals = eigVals[idx]
    eigVecs = eigVecs[:, idx]
    SbxEigVecs = np.dot(PhibX, eigVecs)
    SbxEigVals = np.diag(eigVals)
    Wbx = np.dot(SbxEigVecs, np.linalg.inv(np.sqrt(SbxEigVals)))

    # Diagonalize the between-class scatter matrix (Sb) for Y
    artSby = np.dot(PhibY.T, PhibY)
    
    eigVals, eigVecs = np.linalg.eig(artSby)
    eigVals = np.abs(eigVals)
    zeroEigIdx = np.where(eigVals / np.max(eigVals) < 1e-6)
    eigVals = np.delete(eigVals, zeroEigIdx)
    eigVecs = np.delete(eigVecs, zeroEigIdx, axis=1)
    idx = np.argsort(eigVals)[::-1]
    eigVals = eigVals[idx]
    eigVecs = eigVecs[:, idx]
    SbyEigVecs = np.dot(PhibY, eigVecs)
    SbyEigVals = np.diag(eigVals)
    Wby = np.dot(SbyEigVecs, np.linalg.inv(np.sqrt(SbyEigVals)))

    # Project data to the space where between-class scatter matrices are identity
    r = min(Wbx.shape[1], Wby.shape[1])
    Wbx = Wbx[:, :r]
    Wby = Wby[:, :r]
    Xp = np.dot(Wbx.T, X)
    Yp = np.dot(Wby.T, Y)

    # Unitize the between-set covariance matrix (Sxy)
    Sxy = np.dot(Xp, Yp.T)
    Wcx, s, Wcy = svd(Sxy)
    Wcx = np.dot(Wcx, np.linalg.inv(np.sqrt(np.diag(s))))
    Wcy = np.dot(Wcy, np.linalg.inv(np.sqrt(np.diag(s))))
    Xs = np.dot(Wcx.T, Xp)
    Ys = np.dot(Wcy.T, Yp)
    Ax = np.dot(Wcx.T, Wbx.T)
    Ay = np.dot(Wcy.T, Wby.T)

    return Ax, Ay, Xs, Ys


In [5]:
dca_fuse(X,Y,label)

ValueError: The length of the label must be equal to the number of samples.

In [6]:
X.shape,Y.shape,label.shape

((19, 1101), (19, 1101), (1, 1101))